<a href="https://colab.research.google.com/github/mercyakshitha/FMML_LABS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

question no1:

The accuracy of a model's validation set can be affected by the percentage of data allocated to the validation set in the following ways:

>>Increasing the Percentage of Validation Set:

When you increase the percentage of data in the validation set, you have less training data available. This can lead to a few consequences:

*Pros:
The model may generalize better because it's forced to learn from a smaller training set, which can reduce overfitting.
The validation set may be more representative of the overall data distribution, leading to a more reliable estimate of the model's performance.

*Cons:
Since you have less data for training, the model may not learn as much, and its overall performance might be limited.
If the validation set becomes too large, you risk underfitting the model because it doesn't have enough diverse data to learn from.

>>Reducing the Percentage of Validation Set:

When you reduce the percentage of data in the validation set, you have more data available for training. This can have the following effects:
*Pros:
The model can potentially learn more from the increased training data, leading to better overall performance.
You may be able to train larger and more complex models with a larger training set.

*Cons:
The validation set may not be representative enough to reliably estimate the model's performance. It might not catch overfitting as effectively.
With a small validation set, you might not detect issues like data leakage or model bias.



queastion no 2:

The size of the training and validation sets can affect how well you can predict the accuracy on the test set using the validation set in the following ways:

*Larger Training Set:

When you have a larger training set, your model has more data to learn from, which can lead to better generalization. This means that if your validation set performs well with a large training set, it's a good indicator that your model will likely perform well on the test set.
However, if your training set is too large relative to the validation set, you might not detect overfitting. Your model might perform very well on the training set but poorly on the test set, and the validation set may not provide a reliable estimate of this performance gap.

*Larger Validation Set:

A larger validation set can provide a more reliable estimate of your model's performance because it's a better representation of the overall data distribution. If your model performs well on a large validation set, it's more likely to generalize well to the test set.
However, if your validation set is too large relative to the training set, you might not have enough data to train the model effectively. This can lead to underfitting, where the model doesn't learn the underlying patterns in the data.

*Balanced Split:

It's important to strike a balance between the sizes of the training and validation sets. Common practices include using around 70-80% of the data for training and the remaining 20-30% for validation. This balance allows the model to learn effectively while still providing a reliable estimate of performance.
With a balanced split, you can better predict how your model will perform on the test set based on its performance on the validation set. If the model performs well on validation data, it's a reasonable indicator of good test set performance, assuming the data split is representative.

question no 3:

A commonly used and balanced percentage for reserving the validation set is around 20-30% of your total dataset. This means that you would use approximately 70-80% of your data for training and the remaining 20-30% for validation.

This split strikes a reasonable balance between having enough data for your model to learn effectively during training and having a representative portion of data to evaluate its performance and generalize well to unseen data. However, the optimal split percentage can still vary depending on the specific characteristics of your dataset and the problem you're trying to solve.

Here are a few considerations to keep in mind when determining the split percentage:

Size of the Dataset: If you have a very large dataset, you might be able to allocate a smaller percentage to validation and still have a sufficiently large validation set.

Complexity of the Model: More complex models may require a larger validation set to effectively detect overfitting, so consider increasing the validation set size in such cases.

Data Quality: If your dataset contains noisy or uncertain data, a larger validation set can help provide a more robust estimate of model performance.

Available Computing Resources: Training on a larger portion of the data may require more computational resources. Be sure your infrastructure can handle the chosen split.

question no 4:

Increasing the number of iterations can help mitigate overfitting when dealing with a small training dataset. However, it's essential to strike a balance, as excessively increasing iterations can also lead to overfitting. Regularization techniques like dropout, weight decay, or using a smaller, simpler model architecture may be more effective in such scenarios. Experimentation and monitoring validation performance are key to finding the right balance for your specific dataset and model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


question no 1:

Yes, averaging the validation accuracy across multiple splits of your dataset, such as in k-fold cross-validation, can give more consistent and reliable results compared to relying on a single validation split. This approach helps reduce the impact of randomness and ensures that your model's performance is assessed on a more representative sample of your data. It provides a more robust estimate of your model's generalization performance, which is especially important when dealing with limited data or when you want to have a better understanding of how your model performs on different subsets of the data.



question no 2:

 The accuracy of a test estimate depends on various factors, including the quality and quantity of data, the complexity of the model, and the evaluation techniques used. Generally, using a larger and more diverse dataset for training and evaluation can lead to more accurate estimates of test accuracy. Additionally, using appropriate cross-validation techniques can help provide a more robust estimate of a model's performance.

If you have a specific context or model in mind, please provide more details, and I can offer more tailored advice.


question no 3:

The effect of the number of iterations on estimates depends on the specific algorithm or method being used. In many iterative algorithms, increasing the number of iterations can lead to improved estimates up to a certain point. However, there are diminishing returns, and at some point, further iterations may not significantly improve the estimate or may even lead to overfitting or increased computational cost.

Whether higher iterations result in a better estimate depends on factors like the convergence properties of the algorithm, the quality of the initial estimate, and the trade-off between accuracy and computational resources. It's essential to strike a balance between the number of iterations and the desired level of accuracy in any iterative process.
In some cases, excessive iterations can lead to problems like overfitting, where the algorithm fits the noise in the data rather than the underlying pattern. Therefore, it's often necessary to monitor convergence and use techniques like cross-validation to determine an appropriate number of iterations for a specific problem.

questions no 4:

Increasing the number of iterations can help mitigate overfitting when dealing with a small training dataset. However, it's essential to strike a balance, as excessively increasing iterations can also lead to overfitting. Regularization techniques like dropout, weight decay, or using a smaller, simpler model architecture may be more effective in such scenarios. Experimentation and monitoring validation performance are key to finding the right balance for your specific dataset and model.